# Capítulo 5: Hierarchical Architecture

**Arquitetura Coordinator-Specialist para Sistemas Multi-Agent**  
**Nível:** Intermediário  
**Tempo estimado:** 35-45 minutos

---

## 📖 Sobre Este Capítulo

A **arquitetura hierárquica (Hierarchical)** é um dos padrões fundamentais em sistemas multi-agent. Neste capítulo, você vai dominar o padrão **Coordinator-Specialist** e entender quando ele é a escolha certa.

### O que você vai aprender:

- 🎯 **Padrão Coordinator-Specialist**: Como funciona a delegação hierárquica
- 🏗️ **Implementação com DSPy**: Coordenador + múltiplos especialistas
- ⚖️ **Trade-offs**: Quando usar Hierarchical vs Sequential
- 🔄 **Roteamento dinâmico**: Como o coordenador decide
- 🧪 **Testes práticos**: Validar comportamento do sistema

### Por que Hierarchical?

Na arquitetura Sequential (Cap 4), TODOS os agentes sempre executam. Em Hierarchical, apenas o especialista NECESSÁRIO é chamado.

**Analogia:**
- **Sequential**: Fábrica com esteira (todos processam)
- **Hierarchical**: Hospital com triagem (coordenador → especialista certo)

### Pré-requisitos:

- ✅ Cap 2: DSPy Essentials & Single Agent
- ✅ Cap 4: Sequential/Pipeline Architecture  
- ✅ Compreensão de agentes múltiplos

---

**Vamos começar!** 🚀

## 🎯 Objetivos de Aprendizado

Ao final deste capítulo, você será capaz de:

### 1. **Compreender a Arquitetura Hierarchical**
- Entender o padrão Coordinator-Specialist
- Identificar quando usar hierarquia vs pipeline
- Reconhecer vantagens e limitações

### 2. **Implementar Sistema Hierárquico**
- Criar coordenador que roteia requisições
- Implementar especialistas em domínios específicos
- Integrar coordenador + especialistas

### 3. **Avaliar Trade-offs**
- Comparar Hierarchical vs Sequential
- Analisar custos de coordenação
- Entender impacto em performance e qualidade

### 4. **Decidir Quando Usar**
- Reconhecer cenários ideais para Hierarchical
- Identificar quando outras arquiteturas são melhores
- Balancear complexidade vs benefícios

### 5. **Debugging e Manutenção**
- Rastrear decisões do coordenador
- Isolar problemas por especialista
- Otimizar roteamento

---

**Tempo estimado:** 35-45 minutos  
**Nível de dificuldade:** Intermediário

## 📚 Parte 1: O que é Arquitetura Hierarchical?

### Definição

A **arquitetura hierárquica** organiza agentes em níveis:
- **Nível superior**: Coordenador (decide)
- **Nível inferior**: Especialistas (executam)

## 🏗️ Parte 2: Padrão Coordinator-Specialist (Teoria Completa)

### Anatomia do Coordenador

O **coordenador** é o cérebro do sistema. Suas responsabilidades:

#### 1. **Análise de Requisição**

## ⚖️ Parte 3: Quando Usar vs Quando NÃO Usar

### ✅ Cenários IDEAIS para Hierarchical

#### 1. **Domínios Claramente Separados**

**Exemplo:** Sistema bancário

## 🔄 Hierarchical vs Sequential: Deep Dive

### Comparação Visual

#### Sequential (Cap 4):

In [ ]:
# Imports necessários
import dspy
import os
from datetime import datetime, timedelta
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field
import json
import uuid
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configurar LLM
# IMPORTANTE: Ajuste para seu provedor
lm = dspy.LM('openai/gpt-4o-mini')  # ou groq/llama-3.3-70b-versatile
dspy.configure(lm=lm)

print("✅ Setup completo!")
print(f"📊 LLM configurado: {lm.model}")

In [ ]:
# Data models para nosso sistema de booking de voos

class UserProfile(BaseModel):
    """Perfil do usuário com preferências"""
    name: str
    user_id: str
    email: str
    phone: str
    frequent_flyer_number: Optional[str] = None
    preferences: Dict[str, Any] = Field(default_factory=dict)
    
    class Config:
        json_schema_extra = {
            "example": {
                "name": "João Silva",
                "user_id": "user_001",
                "email": "joao@example.com",
                "phone": "+55-11-99999-9999",
                "preferences": {
                    "preferred_airlines": ["LATAM", "GOL"],
                    "seat_preference": "window",
                    "priority": "price"  # ou "duration", "comfort"
                }
            }
        }

class Flight(BaseModel):
    """Informações de um voo"""
    flight_id: str
    flight_number: str
    departure_airport: str
    arrival_airport: str
    departure_time: str
    arrival_time: str
    duration_minutes: int
    price: float
    available_seats: int
    airline: str = "Default Airlines"
    aircraft_type: str = "Boeing 737"

class Itinerary(BaseModel):
    """Itinerário de viagem"""
    itinerary_id: str
    user_id: str
    flights: List[Flight]
    total_price: float
    booking_date: str
    status: str  # "confirmed", "pending", "cancelled"

print("✅ Data models definidos!")
print(f"📦 Models disponíveis: UserProfile, Flight, Itinerary")

In [ ]:
# Mock database para demonstração

# Banco de usuários
users_db = {
    "Ana": UserProfile(
        name="Ana",
        user_id="user_001",
        email="ana@example.com",
        phone="+55-11-98888-8888",
        frequent_flyer_number="FF12345",
        preferences={
            "preferred_airlines": ["LATAM", "Azul"],
            "seat_preference": "window",
            "priority": "price"  # Ana prefere voos baratos
        }
    ),
    "Carlos": UserProfile(
        name="Carlos",
        user_id="user_002",
        email="carlos@example.com",
        phone="+55-21-97777-7777",
        preferences={
            "preferred_airlines": ["GOL"],
            "seat_preference": "aisle",
            "priority": "duration"  # Carlos prefere voos rápidos
        }
    )
}

# Banco de voos
flights_db = {
    "GRU-SDU": [  # São Paulo → Rio de Janeiro
        Flight(
            flight_id="f001",
            flight_number="LA3000",
            departure_airport="GRU",
            arrival_airport="SDU",
            departure_time="08:00",
            arrival_time="09:05",
            duration_minutes=65,
            price=350.00,
            available_seats=15,
            airline="LATAM",
            aircraft_type="Airbus A320"
        ),
        Flight(
            flight_id="f002",
            flight_number="G3100",
            departure_airport="GRU",
            arrival_airport="SDU",
            departure_time="14:00",
            arrival_time="15:10",
            duration_minutes=70,
            price=280.00,
            available_seats=8,
            airline="GOL",
            aircraft_type="Boeing 737"
        ),
        Flight(
            flight_id="f003",
            flight_number="AD4500",
            departure_airport="GRU",
            arrival_airport="SDU",
            departure_time="11:00",
            arrival_time="12:00",
            duration_minutes=60,
            price=420.00,
            available_seats=12,
            airline="Azul",
            aircraft_type="Embraer E195"
        )
    ]
}

# Bancos para itinerários (serão preenchidos durante booking)
itineraries_db = {}

# Ferramentas para os especialistas

def fetch_flight_info(departure: str, arrival: str, date: str) -> str:
    """Buscar voos disponíveis para uma rota e data."""
    route = f"{departure}-{arrival}"
    flights = flights_db.get(route, [])
    
    if not flights:
        return json.dumps({"error": f"Nenhum voo encontrado para rota {route}"})
    
    flights_data = [flight.model_dump() for flight in flights]
    return json.dumps({
        "flights": flights_data,
        "count": len(flights_data),
        "route": route
    }, ensure_ascii=False, indent=2)

def get_user_info(name: str) -> str:
    """Obter informações do perfil do usuário."""
    user = users_db.get(name)
    if not user:
        return json.dumps({"error": f"Usuário {name} não encontrado"})
    return json.dumps({"user": user.model_dump()}, ensure_ascii=False, indent=2)

def analyze_user_preferences(user_name: str, flights_json: str) -> str:
    """Analisar preferências do usuário e ranquear voos."""
    user = users_db.get(user_name)
    if not user:
        return json.dumps({"error": "Usuário não encontrado"})
    
    # Parse voos
    flights_data = json.loads(flights_json)
    flights = [Flight(**f) for f in flights_data.get("flights", [])]
    
    # Obter preferências
    preferences = user.preferences
    priority = preferences.get("priority", "price")
    
    # Ranquear baseado na prioridade
    if priority == "price":
        ranked = sorted(flights, key=lambda f: f.price)
    elif priority == "duration":
        ranked = sorted(flights, key=lambda f: f.duration_minutes)
    else:
        ranked = flights
    
    # Priorizar companhias aéreas preferidas
    preferred_airlines = preferences.get("preferred_airlines", [])
    if preferred_airlines:
        preferred = [f for f in ranked if f.airline in preferred_airlines]
        others = [f for f in ranked if f.airline not in preferred_airlines]
        ranked = preferred + others
    
    return json.dumps({
        "ranked_flights": [f.model_dump() for f in ranked],
        "recommendation_reason": f"Ranqueado por {priority}, companhias preferidas primeiro",
        "top_recommendation": ranked[0].model_dump() if ranked else None
    }, ensure_ascii=False, indent=2)

def book_flight(user_name: str, flight_id: str, date: str) -> str:
    """Reservar um voo para um usuário."""
    user = users_db.get(user_name)
    if not user:
        return json.dumps({"error": f"Usuário {user_name} não encontrado"})
    
    # Buscar voo
    flight = None
    for route_flights in flights_db.values():
        for f in route_flights:
            if f.flight_id == flight_id:
                flight = f
                break
        if flight:
            break
    
    if not flight:
        return json.dumps({"error": f"Voo {flight_id} não encontrado"})
    
    if flight.available_seats <= 0:
        return json.dumps({"error": "Não há assentos disponíveis"})
    
    # Criar itinerário
    itinerary_id = str(uuid.uuid4())
    confirmation_number = f"CONF{uuid.uuid4().hex[:8].upper()}"
    
    itinerary = Itinerary(
        itinerary_id=itinerary_id,
        user_id=user.user_id,
        flights=[flight],
        total_price=flight.price,
        booking_date=datetime.now().strftime("%Y-%m-%d"),
        status="confirmed"
    )
    
    # Salvar no banco
    itineraries_db[itinerary_id] = itinerary
    flight.available_seats -= 1  # Reduzir assentos disponíveis
    
    return json.dumps({
        "success": True,
        "confirmation_number": confirmation_number,
        "itinerary_id": itinerary_id,
        "flight": flight.model_dump(),
        "total_price": flight.price,
        "message": f"Reserva confirmada! Código: {confirmation_number}"
    }, ensure_ascii=False, indent=2)

print("✅ Mock database e ferramentas criadas!")
print(f"👥 Usuários: {list(users_db.keys())}")
print(f"✈️ Rotas: {list(flights_db.keys())}")

## 🔧 Parte 4: Implementação

### Arquitetura que Vamos Construir

In [ ]:
# Signatures: definem o "contrato" de cada agente

class SearchSpecialistSignature(dspy.Signature):
    """
    Especialista em busca de voos.
    Encontra voos disponíveis baseado em critérios do usuário.
    """
    user_request: str = dspy.InputField(desc="Requisição do usuário")
    departure: str = dspy.InputField(desc="Aeroporto de partida (código IATA)")
    arrival: str = dspy.InputField(desc="Aeroporto de chegada (código IATA)")
    date: str = dspy.InputField(desc="Data do voo (YYYY-MM-DD)")
    
    analysis: str = dspy.OutputField(desc="Análise da busca realizada")
    flights_found: str = dspy.OutputField(desc="Voos encontrados (JSON)")

class RecommendationSpecialistSignature(dspy.Signature):
    """
    Especialista em recomendações de voos.
    Analisa preferências do usuário e ranqueia opções.
    """
    user_request: str = dspy.InputField(desc="Requisição do usuário")
    user_name: str = dspy.InputField(desc="Nome do usuário")
    available_flights: str = dspy.InputField(desc="Voos disponíveis (JSON)")
    
    analysis: str = dspy.OutputField(desc="Análise das preferências e opções")
    recommendation: str = dspy.OutputField(desc="Recomendação final com justificativa")

class BookingSpecialistSignature(dspy.Signature):
    """
    Especialista em reservas de voos.
    Realiza o booking e confirma a reserva para o usuário.
    """
    user_request: str = dspy.InputField(desc="Requisição do usuário")
    user_name: str = dspy.InputField(desc="Nome do usuário")
    flight_id: str = dspy.InputField(desc="ID do voo a reservar")
    date: str = dspy.InputField(desc="Data do voo")
    
    booking_result: str = dspy.OutputField(desc="Resultado da reserva (JSON)")
    confirmation: str = dspy.OutputField(desc="Mensagem de confirmação para o usuário")

class CoordinatorSignature(dspy.Signature):
    """
    Coordenador que analisa a requisição do usuário e decide
    qual especialista deve ser chamado para resolver a tarefa.
    """
    user_request: str = dspy.InputField(desc="Requisição completa do usuário")
    
    required_specialist: str = dspy.OutputField(
        desc="Nome do especialista necessário: 'search', 'recommendation', 'booking', ou 'general'"
    )
    reasoning: str = dspy.OutputField(desc="Razão da escolha do especialista")

print("✅ Signatures definidas!")
print("📋 Disponíveis:")
print("   - SearchSpecialistSignature")
print("   - RecommendationSpecialistSignature")
print("   - BookingSpecialistSignature")
print("   - CoordinatorSignature")

In [ ]:
# Implementação dos agentes especializados

class SearchSpecialist(dspy.Module):
    """
    Especialista em busca de voos.
    Usa ferramentas de busca e analisa resultados.
    """
    def __init__(self):
        super().__init__()
        # ChainOfThought para reasoning estruturado
        self.predictor = dspy.ChainOfThought(SearchSpecialistSignature)
        
    def forward(self, user_request: str, departure: str, arrival: str, date: str):
        # Passo 1: Buscar voos usando ferramenta
        flights_json = fetch_flight_info(departure, arrival, date)
        
        # Passo 2: Analisar resultados com LLM
        result = self.predictor(
            user_request=user_request,
            departure=departure,
            arrival=arrival,
            date=date
        )
        
        # Retornar análise + dados
        return dspy.Prediction(
            analysis=result.analysis,
            flights_found=flights_json
        )

class RecommendationSpecialist(dspy.Module):
    """
    Especialista em recomendações.
    Analisa preferências do usuário e ranqueia voos.
    """
    def __init__(self):
        super().__init__()
        self.predictor = dspy.ChainOfThought(RecommendationSpecialistSignature)
        
    def forward(self, user_request: str, user_name: str, available_flights: str):
        # Passo 1: Analisar preferências e ranquear
        recommendation_json = analyze_user_preferences(user_name, available_flights)
        
        # Passo 2: Gerar recomendação explicativa com LLM
        result = self.predictor(
            user_request=user_request,
            user_name=user_name,
            available_flights=available_flights
        )
        
        # Retornar análise + recomendação ranqueada
        return dspy.Prediction(
            analysis=result.analysis,
            recommendation=recommendation_json
        )

class BookingSpecialist(dspy.Module):
    """
    Especialista em reservas.
    Realiza booking e gera confirmação.
    """
    def __init__(self):
        super().__init__()
        self.predictor = dspy.ChainOfThought(BookingSpecialistSignature)
        
    def forward(self, user_request: str, user_name: str, flight_id: str, date: str):
        # Passo 1: Realizar booking usando ferramenta
        booking_result = book_flight(user_name, flight_id, date)
        
        # Passo 2: Gerar mensagem de confirmação amigável
        result = self.predictor(
            user_request=user_request,
            user_name=user_name,
            flight_id=flight_id,
            date=date
        )
        
        # Retornar resultado + confirmação
        return dspy.Prediction(
            booking_result=booking_result,
            confirmation=result.confirmation
        )

print("✅ Especialistas implementados!")
print("🤖 Agentes criados:")
print("   - SearchSpecialist")
print("   - RecommendationSpecialist")
print("   - BookingSpecialist")

In [ ]:
# Sistema Hierarchical Multi-Agent completo

class HierarchicalMultiAgent(dspy.Module):
    """
    Sistema hierárquico com coordenador e especialistas.
    
    Arquitetura:
                [Coordinator]
                      |
        ┌─────────────┼─────────────┐
        ↓             ↓             ↓
    [Search]    [Recommendation]  [Booking]
    
    O coordenador analisa a requisição e delega para
    o especialista apropriado.
    """
    
    def __init__(self):
        super().__init__()
        
        # Coordenador: decide qual especialista chamar
        self.coordinator = dspy.ChainOfThought(CoordinatorSignature)
        
        # Especialistas: executam tarefas específicas
        self.search_specialist = SearchSpecialist()
        self.recommendation_specialist = RecommendationSpecialist()
        self.booking_specialist = BookingSpecialist()
        
    def forward(self, user_request: str, **kwargs):
        """
        Processa requisição do usuário:
        1. Coordenador analisa e decide especialista
        2. Delega para especialista escolhido
        3. Retorna resultado
        """
        
        # Passo 1: Coordenador decide qual especialista usar
        coordination = self.coordinator(user_request=user_request)
        
        specialist_type = coordination.required_specialist.lower()
        
        # Logging da decisão do coordenador
        print(f"🎯 Decisão do Coordenador: {specialist_type}")
        print(f"💭 Raciocínio: {coordination.reasoning}\n")
        
        # Passo 2: Delegar para especialista apropriado
        
        if "search" in specialist_type:
            # Chamar SearchSpecialist
            result = self.search_specialist(
                user_request=user_request,
                departure=kwargs.get("departure", "GRU"),
                arrival=kwargs.get("arrival", "SDU"),
                date=kwargs.get("date", "2025-12-01")
            )
            return dspy.Prediction(
                specialist="search",
                analysis=result.analysis,
                data=result.flights_found
            )
            
        elif "recommend" in specialist_type:
            # Chamar RecommendationSpecialist
            result = self.recommendation_specialist(
                user_request=user_request,
                user_name=kwargs.get("user_name", "Ana"),
                available_flights=kwargs.get("available_flights", "{}")
            )
            return dspy.Prediction(
                specialist="recommendation",
                analysis=result.analysis,
                data=result.recommendation
            )
            
        elif "book" in specialist_type:
            # Chamar BookingSpecialist
            result = self.booking_specialist(
                user_request=user_request,
                user_name=kwargs.get("user_name", "Ana"),
                flight_id=kwargs.get("flight_id", "f001"),
                date=kwargs.get("date", "2025-12-01")
            )
            return dspy.Prediction(
                specialist="booking",
                confirmation=result.confirmation,
                data=result.booking_result
            )
        
        # Fallback: Coordenador responde diretamente
        return dspy.Prediction(
            specialist="general",
            message="Requisição processada diretamente pelo coordenador"
        )

# Instanciar sistema
hierarchical_system = HierarchicalMultiAgent()

print("✅ Sistema Hierarchical criado!")
print("🏗️ Arquitetura completa pronta para uso")

## 🔄 Como Funciona o Fluxo de Execução

### Anatomia de uma Requisição

#### Exemplo 1: Busca de Voos

## 🧪 Parte 5: Testes do Sistema Hierarchical

### Casos de Teste

Vamos testar 3 cenários diferentes para validar o roteamento:

#### Teste 1: Busca de Voos 🔍
**Objetivo:** Verificar se coordenador escolhe SearchSpecialist

**Input:**

In [ ]:
# Testes do sistema hierarchical

print("=" * 70)
print("🧪 TESTES DO SISTEMA HIERARCHICAL")
print("=" * 70)

# ═══════════════════════════════════════════════════════════
# TESTE 1: Busca de Voos
# ═══════════════════════════════════════════════════════════

print("\n📍 TESTE 1: Busca de Voos")
print("-" * 70)

result1 = hierarchical_system(
    user_request="Preciso encontrar voos de GRU para SDU no dia 15 de dezembro",
    departure="GRU",
    arrival="SDU",
    date="2025-12-15"
)

print(f"✅ Especialista usado: {result1.specialist}")
print(f"\n📊 Análise do especialista:")
print(f"{result1.analysis}")
print(f"\n📦 Dados retornados (primeiros 300 chars):")
print(f"{result1.data[:300]}...")

# ═══════════════════════════════════════════════════════════
# TESTE 2: Recomendação
# ═══════════════════════════════════════════════════════════

print("\n" + "=" * 70)
print("📍 TESTE 2: Recomendação de Voo")
print("-" * 70)

# Primeiro buscar voos para ter dados
flights_result = fetch_flight_info("GRU", "SDU", "2025-12-15")

result2 = hierarchical_system(
    user_request="Qual voo você recomenda para mim? Eu prefiro opções mais baratas.",
    user_name="Ana",  # Ana tem preferência por preço
    available_flights=flights_result
)

print(f"✅ Especialista usado: {result2.specialist}")
print(f"\n📊 Análise do especialista:")
print(f"{result2.analysis}")
print(f"\n💡 Recomendação (primeiros 400 chars):")
print(f"{result2.data[:400]}...")

# ═══════════════════════════════════════════════════════════
# TESTE 3: Reserva (COMENTADO por padrão)
# ═══════════════════════════════════════════════════════════

print("\n" + "=" * 70)
print("📍 TESTE 3: Reserva de Voo")
print("-" * 70)
print("⚠️  NOTA: Teste de booking comentado para não criar reservas reais.")
print("    Descomente o código abaixo para testar booking.")

# DESCOMENTE PARA TESTAR BOOKING:
"""
result3 = hierarchical_system(
    user_request="Quero reservar o voo f002 para o dia 15 de dezembro",
    user_name="Ana",
    flight_id="f002",
    date="2025-12-15"
)

print(f"✅ Especialista usado: {result3.specialist}")
print(f"\n📧 Confirmação:")
print(f"{result3.confirmation}")
print(f"\n📦 Resultado do booking:")
import json
booking_data = json.loads(result3.data)
print(f"  Código de confirmação: {booking_data.get('confirmation_number')}")
print(f"  Preço total: R$ {booking_data.get('total_price')}")
print(f"  Status: {booking_data.get('success')}")
"""

print("\n" + "=" * 70)
print("✅ TESTES CONCLUÍDOS!")
print("=" * 70)

## 📊 Análise dos Resultados

### O que Observar nos Testes

#### 1. **Routing Accuracy** (Precisão do Roteamento)

**Perguntas:**
- ✅ Coordenador escolheu o especialista correto em cada caso?
- ✅ O raciocínio do coordenador faz sentido?
- ✅ Há casos ambíguos que confundem o coordenador?

**Exemplo de sucesso:**

## 🎯 Trade-offs e Conclusões

### Recap: O que Aprendemos

#### ✅ Arquitetura Hierarchical

**Padrão:**